In [1]:
#Import Modules
from logger import setup_custom_logger
import pandas as pd
import requests
import cbsodata
import datetime
pd.options.mode.chained_assignment = None  # default='warn' #We dont care for chained assignments because we dont go back to original df

In [2]:
# Setup Logger
try:
    logger = setup_custom_logger("S002_Average_Sales_Price_New_Construction")
    logger.info('starting')
except:
    logger.exception('logger could not be loaded')
    raise

2021-04-06 14:35:49 INFO     S002_Average_Sales_Price_New_Construction starting


INFO:S002_Average_Sales_Price_New_Construction:starting


In [3]:
try:
    # Get current date information
    now = datetime.datetime.now()
    
    logger.info('datetime loaded')
    yearMin = now.year-3
    yearMax = now.year
    
    # Values you can load
    # now.year
    # now.month
    # now.day
    # now.hour
    # now.minute
except:
    logger.exception('datetime could not be loaded')
    logger.info('set yearmin to a default value')
    # Set default values for fallback
    yearMin = 2010
    yearMax = 2030
    raise

2021-04-06 14:35:49 INFO     S002_Average_Sales_Price_New_Construction datetime loaded


INFO:S002_Average_Sales_Price_New_Construction:datetime loaded


In [4]:
# Dataset 84064NED
dataset_id = '84064NED'
# Table definitions

# Perioden: YYYY%%MM
# PrijsindexVerkoopprijzen_1 = Huizenprijsindex Prijsindex verkoopprijzen
# OntwikkelingTOVVoorgaandePeriode_2 = Ontwikkeling t.o.v. voorgaande periode
# OntwikkelingTOVEenJaarEerder_3 = Ontwikkeling t.o.v. een jaar eerder
# Aantal_4 = Verkochte woningen Aantal
#OntwikkelingTOVVoorgaandePeriode_5 =  Ontwikkeling t.o.v. voorgaande periode (woningen)
#OntwikkelingTOVEenJaarEerder_6 = Ontwikkeling t.o.v. een jaar eerder (woningen)
#GemiddeldeVerkoopprijs_7 = Gemiddelde verkoopprijs
#TotaleWaardeVerkoopprijzen_8 = Totale waarde verkoopprijzen

In [5]:
try:
    logger.info(f'Retrieve data from dataset {dataset_id}')
    df = pd.DataFrame(cbsodata.get_data(
    f'{dataset_id}',
    filters=f"substring(Perioden,0,4) ge '{yearMin}'")) # These spaces need to be there, they are part of the value..
except:
    logger.exception('error loading data from CBS Statline')
    raise

2021-04-06 14:35:50 INFO     S002_Average_Sales_Price_New_Construction Retrieve data from dataset 84064NED


INFO:S002_Average_Sales_Price_New_Construction:Retrieve data from dataset 84064NED


In [6]:
# Remove yearly average and totale. Also multiply total value to make true million number
try:
    df["TotaleWaardeVerkoopprijzen_8"] = df["TotaleWaardeVerkoopprijzen_8"] * 1000000
    df = df[df['Perioden'].map(len) > 4 ]
    df = df.loc[(df['SoortKoopwoning'] != "Totaal koopwoningen")]
except:
    logger.exception('Perioden filter could not be applied')
    raise

In [7]:
# Date formatting and quarter format
try:
    df["Perioden"] = df["Perioden"].str.replace(" 1e kwartaal", "-03-01")
    df["Perioden"] = df["Perioden"].str.replace(" 2e kwartaal", "-06-01")
    df["Perioden"] = df["Perioden"].str.replace(" 3e kwartaal", "-09-01")
    df["Perioden"] = df["Perioden"].str.replace(" 4e kwartaal", "-12-01")
    df['Perioden'] = pd.to_datetime(df["Perioden"]).dt.date
except:
    logger.exception('Columns could not be changed to monthly numbers or formatted to different date')
    raise

In [8]:
try:
    df = df.groupby(['Perioden','SoortKoopwoning'])[['PrijsindexVerkoopprijzen_1', 'OntwikkelingTOVVoorgaandePeriode_2','OntwikkelingTOVEenJaarEerder_3', 'Aantal_4','OntwikkelingTOVVoorgaandePeriode_5','OntwikkelingTOVEenJaarEerder_6','GemiddeldeVerkoopprijs_7','TotaleWaardeVerkoopprijzen_8']].apply(sum)
except:
    logger.exception('Grouping data failed')
    raise

In [9]:
# Export dataFrame to Excel file
try:
    df.to_csv("output/S002_Average_Sales_Price_New_Construction.csv")  
except:
    logger.exception('dataFrame could not be exported to output folder')